sympy tactics
simp
rw
natural number game

progress and preservation


https://leanprover.github.io/theorem_proving_in_lean4/induction_and_recursion.html

In [16]:
from kdrag.all import *

import kdrag.theories.datatypes as dt
Nat = kd.Inductive("Nat")
Nat.declare("Z")
Nat.declare("S", ("pred", Nat))
Nat = Nat.create()
x,y,z = smt.Consts("x y z", Nat)
add = smt.Function("add", Nat, Nat, Nat)
add = kd.define("add", [x,y], smt.If(x.is_Z, y, Nat.S(add(x.pred, y))))

l = kd.Lemma(smt.ForAll([x], add(x, Nat.Z) == x))
[
    x1 := l.intros(),
    l.induct(x1),
    #l.z3simp(),
    l.auto(by=[add.defn]),
    #l.z3simp(),
    a := l.intros(),
]
l.auto(by=[add.defn])
l.qed()


|- ForAll(x, add(x, Z) == x)

In [68]:
add_z_l = kd.kernel.lemma(smt.ForAll([x], add(Nat.Z, x) == x), by=[add.defn])
add_s_l = kd.kernel.lemma(smt.ForAll([x,y], add(Nat.S(x),y) == Nat.S(add(x,y))), by=[add.defn])


induct is doing to much.
It seems like kernel.lemma needs to be used more often. lemma is too clogged with junk,
Or do pruning in lemma.
rewriting is happening only at one position which feels weird and uncotrolled.


In [19]:
l = kd.Lemma(smt.ForAll([x], add(x, Nat.Z) == x))
x1 = l.intros()
l.induct(x1)
l.auto(by=[add.defn])
l.auto(by=[add.defn])
add_Z_r = l.qed()

l = kd.Lemma(smt.ForAll([x,y], add(x, Nat.S(y)) == Nat.S(add(x, y))))
x1,y1 = l.intros()
l.induct(x1)
l.rw(add.defn)
l.z3simp()
l.rw(add.defn)
l.z3simp()
l.auto()
z1 = l.fixes()
l.auto(by=[add.defn])
add_s_r = l.qed()

l = kd.Lemma(smt.ForAll([x,y], add(x, y) == add(y, x)))
x1,y1 = l.intros()
l.induct(x1)
l.auto(by=[add.defn, add_Z_r])
z1 = l.fixes()
l.auto(by=[add.defn, add_s_r])
add_comm = l.qed()

l = kd.Lemma(smt.ForAll([x,y,z], add(x, add(y, z)) == add(add(x, y), z)))
x1,y1,z1 = l.intros()
l.induct(x1)
l.rw(add_z_l)
l.rw(add_z_l)
l.auto()
l.auto(by=[add.defn, add_comm])
add_assoc = l.qed()
#l.auto(by=[add.defn])



NameError: name 'add_z_l' is not defined

In [6]:
from kdrag.all import *
import kdrag.theories.datatypes as dt

Nat = dt.Inductive("Nat")
Nat.declare("Z")
Nat.declare("S", ("pred", Nat))
#Nat = Nat.create()
Nat = Nat.create()
x = smt.Const("x", Nat)
x.induct()



|- ForAll(P!211,
       Implies(And(P!211[Z],
                   ForAll(a!212,
                          Implies(P!211[a!212],
                                  P!211[S(a!212)]))),
               P!211[x]))

In [24]:
from z3 import *
IntVal(2) + IntVal(3) == IntVal(5)
type(IntVal(5))
type(IntVal(2) + IntVal(3))


def Eq(x,y):
    """Python __eq__ resolution rules flips the order if y is a subclass of x. This function corrects that."""
    e = (x == y)
    if type(x) != type(y) and issubclass(type(y), type(x)):
        return e.decl()(e.arg(1), e.arg(0))
    else:
        return e

assert Eq(IntVal(2) + IntVal(3), IntVal(5)).arg(1).eq(IntVal(5))
Eq(IntVal(5), IntVal(2) + IntVal(3))
x = Int("x")
y = Int("y")
assert Eq((x >= 14), z3.Exists([y], x == 2 * y)).arg(0).eq(x >= 14)
assert Eq((x >= 14), x >= 13).arg(0).eq(x >= 14)


In [21]:
type(IntVal(5))

z3.z3.IntNumRef

In [11]:
from kdrag.all import *
List = kd.Inductive("List<Int>", strict=False)
List.declare("Nil")
List.declare("Cons", ("hd", smt.IntSort()), ("tl", List))
List = List.create()
x,y,z = smt.Consts("x y z", List)

append = smt.Function("append", List, List, List)
append = kd.define("append", [x,y], smt.If(x.is_Nil, y, List.Cons(x.hd, append(x.tl, y))))
kd.notation.add.register(List, append)

a = smt.Int("a")
append_nil_x = kd.lemma(smt.ForAll([x], append(List.Nil, x) == x), by=[append.defn])
append_cons_x = kd.lemma(smt.ForAll([a,y,z], List.Cons(a, y) + z == List.Cons(a, y + z)), by=[append.defn])

l = kd.Lemma(smt.ForAll([x], append(x, List.Nil) == x))
_x = l.intros()
l.induct(_x)
l.auto(by=[append.defn])
_a, = l.fixes()
l.intros()
l.auto(by=append.defn)
l.qed()
#l = kd.Lemma(smt.ForAll([x,y,z], append(List.Cons(a, y), z) == List.Cons(a, append(y, z))))
#l

l = kd.Lemma(smt.ForAll([x,y,z], append(x, append(y, z)) == append(append(x, y), z)))
_x, _y, _z = l.intros()
l.induct(_x)
l.auto(by=[append_nil_x])
l.auto(by=[append_cons_x])
append_assoc = l.qed()






In [23]:
dir(smt.Or(x==x,x==x,x==x))

['__add__',
 '__and__',
 '__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lt__',
 '__matmul__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '__xor__',
 '_repr_html_',
 'arg',
 'as_ast',
 'ast',
 'children',
 'ctx',
 'ctx_ref',
 'decl',
 'defn',
 'eq',
 'from_string',
 'get_id',
 'hash',
 'induct',
 'num_args',
 'params',
 'serialize',
 'sexpr',
 'sort',
 'sort_kind',
 'translate',
 'use_pp',
 'wf']

In [6]:
%%timeit
l = kd.lemma(smt.ForAll([x], append(List.Nil, x) == x), by=[append.defn])

19.9 ms ± 2.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%%timeit
l = kd.kernel.lemma(smt.ForAll([x], append(List.Nil, x) == x), by=[append.defn])

12.9 ms ± 894 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# inductive relation

You can split these things up in lean also. Is this awkward?

```
inductive evidence where
    le_n : evidence
    le_s : evidence -> evidence

def le n m : evidence -> Type := 
  | le_n => n = n
  | le_s ev => ∃ m', m = m' + 1 ∧ le n' m' ev

def le n m : evidence -> Bool := 
  | le_n => n == n
  | le_s ev => is_succ m and le n' (pred m) ev

e : evidence -> p : le n m e -> something

def le n m := exists ev, le' n m ev

```
But if I can poroduce the evidence as a decision procedure...


If I don't minify using evidence.
Consider even and odd defined minikanren style.
A model of either is then all true, or actual minimal
even(x) -> odd(x+1)  
But then in model where even = True, odd doesn't have to bve true of all x. Or the model could be x = 1, even=True, odd=actualodds. Then state is false. Too many models.
even(x) = exists y, x = 2*y is fine though. y is not to disimilar form inductive structure of y

likewise for le n m. exists z, m = n + z


In [ ]:

def Inductive(name):
    if name in records:
        raise Exception("Record already defined", name)
    dt = smt.Datatype(name)
    oldcreate = dt.create
    # TODO: Should I make inductive predicates possible?
    # olddeclare = dt.declare
    # preds = []

    def create():
        dt = oldcreate()
        x = smt.FreshConst(dt, prefix="x")
        kd.notation.induct.register(dt, lambda x: induct_inductive(dt, x=x))
        # kd.define(name, [x], smt.BoolSort(), lambda P: induct(dt, x=P))
        """for i, pred in enumerate(preds):
            if pred is not None:
                smt.Implies(dt.recognizer(i)(x), pred(x))
        """
        records[name] = dt
        return dt

    """
    def declare(*args, pred=None):
        olddeclare(*args)
        #preds.append(pred)
    """
    dt.create = create
    return dt

In [90]:
le = kd.define("le", [x,y], smt.Exists([z], z + x == y))
l = kd.Lemma(smt.ForAll([x], le(x,x)))
x1 = l.intros()
l.unfold(le)
l.exists(Nat.Z)
l.auto(by=[add.defn])


NotImplementedError: 

In [ ]:

leEv = dt.Inductive("leEv")
leEv.declare("le_n")
leEv.declare("le_s", ("pred", leEv))
leEv = leEv.create()
ev = smt.Const("ev", leEv)
le = kd.define("le", [n,m,ev], # open evidence form 
               smt.If(ev.is_le_n, n == 0,
                      le(n.pred, m.pred, ev.pred)))
le = kd.define("le", [n,m], # closed evidence form
               smt.Exists([ev], 
               smt.If(ev.is_le_n, n == 0,
                      le(n.pred, m.pred)))_




# even odd



In [1]:
from kdrag.all import *
import kdrag.theories.int as int_
from kdrag.theories.int import even, odd
x,y,z = smt.Ints("x y z")
even_odd = kd.lemma(smt.ForAll([x], even(x) == odd(x+1)), by=[even.defn, odd.defn])
odd_even = kd.lemma(smt.ForAll([x], odd(x) == even(x-1)), by=[even.defn, odd.defn])

l = kd.Lemma(smt.ForAll([x], even(x) == even(x+2)))
_x = l.fixes()
l.unfold(even)
l.unfold(even)
l.split()
l.intros()
_y = l.einstan(0)
l.exists(_y-1)
l.auto()
l.intros()
_y = l.einstan(0)
l.exists(_y+1)
l.auto()
even_2 = l.qed()

l = kd.Lemma(smt.ForAll([x], odd(x) == even(x+1)))
_x = l.fixes()
l.rw(odd_even)
l.simp()
l.auto(by=[even.defn, even_2])
odd_even_1 = l.qed()
#l.unfold(odd, even)

odd_2 = kd.lemma(smt.ForAll([x], odd(x) == odd(x+2)), by=[odd_even_1, even_2])


Admitting lemma ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z))
Admitting lemma ForAll(x, Implies(x >= 0, sqrt(x) >= 0))
Admitting lemma ForAll(x, Implies(x >= 0, sqrt(x)**2 == x))
Admitting lemma ForAll(x, Implies(x >= 0, sqrt(sqr(x)) == x))


Hmm actually kind of fiddly.
How 

In [4]:
l = kd.Lemma(smt.ForAll([x], even(x) != odd(x)))
_x = l.fixes()
l.induct(_x)
l.auto(by=[even.defn, odd.defn])
_n = l.fixes()
l.intros()
#l.rw(odd_even_1)
#l.rw(kd.kernel.instan([_n], even_odd))
#l.rw(odd_even,rev=True)
#l.unfold(even,odd)

#l.split()
#l.induct(x)


[And(n!385 >= 0, Not(even(n!385) == odd(n!385)))] ?|- Not(even(1 + n!385) == odd(1 + n!385))

In [41]:
kd.lemma(smt.ForAll([x], odd(x) == odd(x+2)), by=[even.defn, odd.defn, even_2])

LemmaError: ('lemma', ForAll(x, odd(x) == odd(x + 2)), [|- ForAll(x, even(x) == (Exists(y, x == 2*y))), |- ForAll(x, odd(x) == (Exists(y, x == 2*y + 1))), |- ForAll(x, even(x) == even(x + 2))], unknown)

# limits


In [8]:
from kdrag.all import *
f = smt.Const("f", real.RFun)
x,y = smt.Reals("x y")
l = kd.Lemma(kd.QForAll([f,x], real.has_lim_at(f,x,f[x]), real.cont_at(f,x)))
_f, _x, = l.fixes()
l.intros()
l.unfold(real.has_lim_at, at=0)
l.unfold(real.cont_at)
_eps = l.fixes()
l.intros()
# I need to instantiate with _eps
# _delta = l.einstan(0)
# l.exists(_delta)
# l.auto()  #maybe. I might need to unfold or use property of abs that abs(x) == 0 -> x == 0 

[ForAll(eps,
       Implies(0 < eps,
               Exists(delta,
                      And(delta > 0,
                          ForAll(x,
                                 Implies(And(0 <
                                        absR(x - x!202),
                                        absR(x - x!202) <
                                        delta),
                                        absR(f!201[x] -
                                        f!201[x!202]) <
                                        eps)))))), eps!208 > 0] ?|- Exists(delta,
       And(delta > 0,
           ForAll(y,
                  Implies(absR(x!202 - y) < delta,
                          absR(f!201[x!202] - f!201[y]) <
                          eps!208))))

In [39]:
from kdrag.all import *
x = smt.Real("x")

l = kd.Lemma(real.has_lim_at(smt.Lambda([x], x), 0, 0))
l.unfold(real.has_lim_at)
_eps = l.fixes()
l.intros()
l.exists(_eps)
l.auto()
l.qed()

l = kd.Lemma(real.has_lim_at(smt.Lambda([x], x / 2), 0, 0))
l.unfold(real.has_lim_at)
_eps = l.fixes()
l.intros()
l.exists(_eps*2)
l.split()
l.auto()
l.fixes()
for i in range(2):
    l.unfold(real.abs)
l.simp()
l.auto(defns=False)
l.qed()
#l.unfold(real.abs)
#l.auto()
#l.qed()

|- has_lim_at(Lambda(x, x/2), 0, 0)

In [ ]:
l = kd.Lemma(real.has_lim_at(smt.Lambda([x], x / 2), 0, 0))
l.unfold(real.has_lim_at)
_eps = l.fixes()
l.intros()
l.exists(_eps*2)

In [16]:
l = kd.lemma(real.has_lim_at(smt.Lambda([x], x), 0, 0), by=[real.has_lim_at.defn, real.abs.defn])

LemmaError: ('lemma', has_lim_at(Lambda(x, x), 0, 0), [|- ForAll([f, p, L],
       has_lim_at(f, p, L) ==
       (ForAll(eps,
               Implies(0 < eps,
                       Exists(delta,
                              And(delta > 0,
                                  ForAll(x,
                                        Implies(And(0 <
                                        absR(x - p),
                                        absR(x - p) < delta),
                                        absR(f[x] - L) < eps)))))))), |- ForAll(x, absR(x) == If(x >= 0, x, -x))], unknown)

# derivstive of tan(x)



In [95]:
from kdrag.all import *
Real = smt.RealSort()
Poly = dt.Inductive("Poly")
Poly.declare("X")
Poly.declare("real", ("val", Real))
Poly.declare("add", ("add1", Poly), ("add2", Poly))
Poly.declare("mul", ("mul1", Poly), ("mul2", Poly))
Poly = Poly.create()

kd.notation.add.register(Poly, Poly.add)
kd.notation.mul.register(Poly, Poly.mul)
p = smt.Const("p", Poly)
apply = smt.Function("apply", Poly, Real, Real)
apply = kd.define("apply", [p,x], 
                  kd.cond((p.is_X, x),
                          (p.is_real, p.val),
                          (p.is_add, apply(p.add1, x) + apply(p.add2, x)),
                          (p.is_mul, apply(p.mul1, x) * apply(p.mul2, x))))

kd.Lemma(smt.ForAll([p,q], apply(p + q, x) == apply(p, x) + apply(q, x)))



AttributeError: 'DatatypeSortRef' object has no attribute 'is_int'

In [78]:
import kdrag.theories.real as real
real.deriv
real.deriv_add
real.deriv_mul

|- ForAll([f, g],
       deriv(mul(f, g)) ==
       add(mul(deriv(f), g), mul(f, deriv(g))))

In [89]:
X = real.X
#kd.Lemma(real.deriv(X*X) == 2*X)
real.deriv(X*X)
real.const(2)*X

mul(const(2), ident)

In [ ]:
deriv(tan(X)) == sec(X)**2    # Derivative of tan(x) is sec(x)^2                                                                                      

# Calculating compiler